In [1]:
!pip install 'qdrant-client[fastembed]' --upgrade

  Attempting uninstall: qdrant-client
    Found existing installation: qdrant-client 1.12.1
    Uninstalling qdrant-client-1.12.1:
      Successfully uninstalled qdrant-client-1.12.1
  Attempting uninstall: fastembed
    Found existing installation: fastembed 0.3.6
    Uninstalling fastembed-0.3.6:
      Successfully uninstalled fastembed-0.3.6


In [2]:
QDRANT_HOST="https://3bdbdcd2-6854-48ab-b5c4-c22a0da98431.europe-west3-0.gcp.cloud.qdrant.io"
QDRANT_API_KEY="eR-0gtWq9ljNyq1b4dFK0sGn9xXpbOmp-UR7J_fEl2ia11ckAWzijw"
QDRANT_PORT=6333

In [3]:
from qdrant_client import AsyncQdrantClient, QdrantClient

# client = QdrantClient(path="path/to/db")  # Persists changes to disk
# or
client = QdrantClient(url=QDRANT_HOST, port=QDRANT_PORT, api_key=QDRANT_API_KEY)

In [4]:
# Prepare your documents, metadata, and IDs
docs = ["Qdrant has Langchain integrations", "Qdrant also has Llama Index integrations"]

client.add(
    collection_name="demo_collection",
    documents=docs,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/133M [00:00<?, ?B/s]

['fe214fbc3a114e22a6447e65d76e404a', 'e74b2308c90e4e728d6fa01c8c5af62e']

In [12]:
search_result = client.query_points(
    collection_name="test_collection",
)
print(search_result)

points=[ScoredPoint(id=1, version=0, score=0.0, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=0, score=0.0, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=0.0, payload={'city': 'Moscow'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=4, version=0, score=0.0, payload={'city': 'New York'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=5, version=0, score=0.0, payload={'city': 'Beijing'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.0, payload={'city': 'Mumbai'}, vector=None, shard_key=None, order_value=None)]


In [13]:
search_result.points

[ScoredPoint(id=1, version=0, score=0.0, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=2, version=0, score=0.0, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=3, version=0, score=0.0, payload={'city': 'Moscow'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=4, version=0, score=0.0, payload={'city': 'New York'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=5, version=0, score=0.0, payload={'city': 'Beijing'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=6, version=0, score=0.0, payload={'city': 'Mumbai'}, vector=None, shard_key=None, order_value=None)]

In [7]:
type(search_result)

qdrant_client.http.models.models.QueryResponse

In [6]:
from qdrant_client.http.models import Distance, VectorParams

if not client.collection_exists("test_collection"):
    client.create_collection(
            collection_name="test_collection",
            vectors_config=VectorParams(size=8, distance=Distance.DOT),
    )

In [7]:
from qdrant_client.http.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74, 0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.05, 0.61, 0.76, 0.74, 0.05, 0.61, 0.76, 0.74], payload={"city": "London"}),
        PointStruct(id=3, vector=[0.05, 0.61, 0.76, 0.74, 0.05, 0.61, 0.76, 0.74], payload={"city": "Moscow"}),
        PointStruct(id=4, vector=[0.05, 0.61, 0.76, 0.74, 0.05, 0.61, 0.76, 0.74], payload={"city": "New York"}),
        PointStruct(id=5, vector=[0.05, 0.61, 0.76, 0.74, 0.05, 0.61, 0.76, 0.74], payload={"city": "Beijing"}),
        PointStruct(id=6, vector=[0.05, 0.61, 0.76, 0.74, 0.05, 0.61, 0.76, 0.74], payload={"city": "Mumbai"}),
    ]
)
print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>


In [9]:
search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.18, 0.81, 0.75, 0.12, 0.12, 0.69, 0.58, 0.54],
    limit=1
)
print(search_result)

[ScoredPoint(id=1, version=0, score=2.4292, payload={'city': 'Berlin'}, vector=None, shard_key=None, order_value=None)]


In [11]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue

search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.18, 0.81, 0.75, 0.12, 0.12, 0.69, 0.58, 0.54],
    query_filter=Filter(
        must=[
            FieldCondition(
                key="city",
                match=MatchValue(value="London")
            )
        ]
    ),
    limit=1
)
print(search_result)

[ScoredPoint(id=2, version=0, score=2.4292, payload={'city': 'London'}, vector=None, shard_key=None, order_value=None)]
